In [1]:
import csv
import time
import numpy as np

from sklearn.cross_validation import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape
from keras.layers import LSTM

Using Theano backend.


In [2]:
def read_data(filename):
    data = {}
    with open(filename, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in reader:
            block = row[-1][1:]
            if block.isdigit():
                block = int(block)
            del row[-1]
            del row[0]
            if not block in data:
                data[block] = {}
                data[block]['gyr'] = {}
                data[block]['acc'] = {}
                data[block]['gyr']['x'] = []
                data[block]['gyr']['y'] = []
                data[block]['gyr']['z'] = []
                data[block]['acc']['x'] = []
                data[block]['acc']['y'] = []
                data[block]['acc']['z'] = []
            frow = np.array(map(float, row))
            data[block]['acc']['x'].append(np.array(frow[range(0,len(row),6)]))
            data[block]['acc']['y'].append(np.array(frow[range(1,len(row),6)]))
            data[block]['acc']['z'].append(np.array(frow[range(2,len(row),6)]))
            data[block]['gyr']['x'].append(np.array(frow[range(3,len(row),6)]))
            data[block]['gyr']['y'].append(np.array(frow[range(4,len(row),6)]))
            data[block]['gyr']['z'].append(np.array(frow[range(5,len(row),6)]))
    return data

In [3]:
def array_from_dict(data):
    X = []
    y = []
    for block in data:
        for ind in range(len(data[block]['acc']['x'])):
            y.append(block)
            X.append(np.dstack((data[block]['acc']['x'][ind], data[block]['acc']['y'][ind], data[block]['acc']['z'][ind], 
                                data[block]['gyr']['x'][ind], data[block]['gyr']['y'][ind], data[block]['gyr']['z'][ind])))
    
    return X, np.array(y)

### Start from here

In [4]:
data1 = read_data('data1.csv')  # Your data
data2 = read_data('data2.csv')  # My data (I dont have a gyroscope)

In [5]:
X, y = array_from_dict(data1)  # X is list of all sequences, each sequence has shape (length, 6); y is a target

In [6]:
b = np.zeros((len(y), 16))
b[np.arange(len(y)), y-1] = 1
y = b  # One hot encoded y

In [7]:
np.random.seed(1)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
model = Sequential()  # Creates a model
model.add(LSTM(120, input_dim=6))  # Recurental layer has 120 neurons
model.add(Dense(16, activation='sigmoid'))  # Kind of output layer

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
print(model.summary())

for seq, label in zip(X_train, y_train):  # I train the model one by one sequence, coz they have different lengths
    label = label.reshape((1, -1))
    model.fit(seq, label, nb_epoch=5)
    time.sleep(0.5)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_2 (LSTM)                    (None, 120)           60960       lstm_input_2[0][0]               
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 16)            1936        lstm_2[0][0]                     
Total params: 62896
____________________________________________________________________________________________________
None
Epoch 1/5
1/1 [==============================] - 0s - loss: 2.8458 - acc: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s - loss: 2.5136 - acc: 1.0000
Epoch 3/5
1/1 [==============================] - 0s - loss: 2.3109 - acc: 1.0000
Epoch 4/5
1/1 [==============================] - 0s - loss: 2.1465 - acc: 1.0000
Epoch 5/5
1/1 [==========================

In [15]:
scores = []
for seq, label in zip(X_test, y_test):
    label = label.reshape((1, -1))
    score = model.evaluate(seq, label)
    print score
    scores.append(score[1])
print 'Mean score:', np.mean(scores)

1/1 [==============================] - 0s
[0.52758270502090454, 1.0]
1/1 [==============================] - 0s
[0.90769988298416138, 1.0]
1/1 [==============================] - 0s
[0.05869036540389061, 1.0]
1/1 [==============================] - 0s
[0.33945000171661377, 1.0]
1/1 [==============================] - 0s
[7.6141667366027832, 0.0]
1/1 [==============================] - 0s
[1.5239888429641724, 0.0]
1/1 [==============================] - 0s
[3.3096928596496582, 0.0]
1/1 [==============================] - 0s
[6.2167224884033203, 0.0]
1/1 [==============================] - 0s
[1.82541823387146, 0.0]
1/1 [==============================] - 0s
[0.16066369414329529, 1.0]
1/1 [==============================] - 0s
[0.9496457576751709, 1.0]
1/1 [==============================] - 0s
[5.9912905693054199, 0.0]
1/1 [==============================] - 0s
[0.086337745189666748, 1.0]
1/1 [==============================] - 0s
[1.059819221496582, 1.0]
1/1 [==============================] - 0s
[0